In [1]:
import os
import pandas as pd
import numpy as np
from math import pi

from math import sqrt
from math import atan2
from math import pi

In [2]:
def columns(df, d, for_merge, v_col, u_col):
    windDir_list  = []
    windSpd_list = []
    windDir_degrees_list = []

    for col in df: 
        v = col[v_col]
        u = col[u_col]
        windDir1 = (270-atan2(v,u)*180/pi)%360      
        windDir_list.append(windDir1)
        windSpd1 = sqrt(u**2+v**2) 
        windSpd_list.append(windSpd1)
    windDir = np.array(windDir_list)
    windSpd = np.array(windSpd_list)

    def degToCompass(num):
            val=int((num/22.5)+.5)
            arr=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
            dir = arr[(val % 16)]
            windDir_degrees_list.append(dir)

    for num in windDir:
        degToCompass(num)

    windDir_degrees = np.array(windDir_degrees_list)

    final = pd.DataFrame({f'M_Spd_{d}m': windSpd, f'M_Direction_{d}m': windDir_degrees, f'M_Dir_{d}m': windDir})
    for_merge[f'{d}'] = final 

In [3]:
directory = "D:/ИТМО/Python/rozes/model"       
for filename in os.listdir(directory):
    if 'csv' in filename:        
        for_merge = {}
        path = f'{directory}/{filename}'
        data = pd.read_csv(path, sep = ';', decimal = ',', index_col = 0)
        df = pd.DataFrame(data)
        df.columns = df.columns.str[:11]
        df = df.iloc[:,1:]
        name = path.split('/')[-1][:-4]
        
        df = df.to_numpy()
        if df.shape[1] < 8:  # если кол-во колонок в numpy array 
            columns(df, 3, for_merge, 0, 3)
            columns(df, 12, for_merge, 1, 4)
            columns(df, 25, for_merge, 2, 5)
        else:
            columns(df, 3, for_merge, 0, 4)
            columns(df, 12, for_merge, 1, 5)
            columns(df, 25, for_merge, 2, 6)
            columns(df, 47, for_merge, 3, 7)

        full = pd.concat(for_merge.values(), axis=1)
        full.to_csv(f"D:/ИТМО/Python/rozes/model/prepared_model/prep_{name}.csv", sep = ';', decimal = ',')